In [380]:
from datetime import datetime
from tkinter import messagebox
import tkinter as tk
import pandas as pd
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore")

In [381]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir('bases/'):
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(fecha_list[0] + '.' + fecha_list[1] + '.' + fecha_list[2], '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

'02.09.2024'

In [382]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

La carpeta ya existe: reporte\2024.09.03


'reporte\\2024.09.03'

In [383]:
base_pagos_path = 'bases/Base Pagos ' + fecha + '.xlsx'
asignacion_path = 'bases/base_asignacion.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = file_path + '/MONOPRODUCTO ' + fecha + '.xlsx'
multiproducto = file_path + '/MULTIPRODUCTO ' + fecha + '.xlsx'
reactiva = file_path + '/REACTIVA ' + fecha + '.xlsx'
no_enviados = file_path + '/NO ENVIADOS ' + fecha + '.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)

bases/Base Pagos 02.09.2024.xlsx
bases/base_asignacion.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.09.03\MONOPRODUCTO 02.09.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.09.03\MULTIPRODUCTO 02.09.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.09.03\REACTIVA 02.09.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.09.03\NO ENVIADOS 02.09.2024.xlsx


In [384]:
#################### BASE DE PAGOS ####################

In [385]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(114, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-08-31,24693403,248,832,3000.00,PEN,-OMAR NICEFORO VENTURA CUEVA,P
1,11,2024-08-31,26255158,248,879,500.00,PEN,CARLOS GUILLERMO MU･OZ MAZUELOS 01/08/24,P
2,11,2024-08-31,21044214,248,970,200.00,PEN,CARMEN ISABEL GIRALDO AYALA....,P
3,11,2024-09-02,26731386,248,597,8000.00,PEN,BERTHA UNTIVEROS SUAREZ 45766607,P
4,11,2024-09-02,24153942,248,123,231.51,PEN,001101835002240287/,P


In [386]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base = df_base.rename(columns={'CODCEN': 'CC'})

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(114, 10)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FECHA_ENVIO,ID_RESPONSABLE
0,2024-08-31,24693403,00248,832,3000.00,PEN,-OMAR NICEFORO VENTURA CUEVA,P,03-Sep,MIV
1,2024-08-31,26255158,00248,879,500.00,PEN,CARLOS GUILLERMO MU･OZ MAZUELOS 01/08/24,P,03-Sep,MIV
2,2024-08-31,21044214,00248,970,200.00,PEN,CARMEN ISABEL GIRALDO AYALA....,P,03-Sep,MIV
3,2024-09-02,26731386,00248,597,8000.00,PEN,BERTHA UNTIVEROS SUAREZ 45766607,P,03-Sep,MIV
4,2024-09-02,24153942,00248,123,231.51,PEN,001101835002240287/,P,03-Sep,MIV


In [387]:
df_base.isnull().sum()

FECHA             0
CC                0
CLAVSERV          0
CENTROPAGO        0
IMPORTE           0
MONEDA            0
NOMBRE            0
ESTADO            0
FECHA_ENVIO       0
ID_RESPONSABLE    0
dtype: int64

In [388]:
#################### ASIGNACION ####################

In [389]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base de asignación?", icon='warning')
if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path)


In [390]:
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

(242511, 9)
codigo               int64
ID_FLAG              int64
cont_18              int64
nombre              object
TIPO_CARTERA        object
TIPO_FONDO          object
clave               object
AGENCIA CORRECTA    object
ID_VTA.             object
dtype: object


,codigo,ID_FLAG,cont_18,nombre,TIPO_CARTERA,TIPO_FONDO,clave,AGENCIA CORRECTA,ID_VTA.
0,30739481,2,1100109600055161,DINA CHOQUE FERNANDEZ,UNSECURED,NaN,BC,MORNESE MORA,-
1,30739481,2,1100109600057679,DINA CHOQUE FERNANDEZ,UNSECURED,NaN,BC,MORNESE MORA,-
2,21874900,4,1100505000360222,JULIA ANGELICA SOTO COTRINA,UNSECURED,NaN,BC,ASESCOM RJ,-
3,30293392,1,1101305002987960,INNOVA FASHION GROUPSAC,UNSECURED,NaN,BC,CLASA MORA,-
4,20687681,1,1101749600365104,ELIZABETH HAYDEE CORDOVA AMAYA,UNSECURED,NaN,BC,RodriguezAngobaldo,-


In [391]:
#################### BACKUPS ####################

In [392]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

Base (114, 10)
Asignacion (242511, 9)


In [393]:
base_count = df_base_test.shape[0]
base_count

114

In [394]:
def actualizar_tipo_cartera(grupo: pd.DataFrame) -> pd.DataFrame:
    if 'UNSECURED' in grupo['TIPO_CARTERA'].values:
        grupo['TIPO_CARTERA'] = 'UNSECURED'
    return grupo

In [395]:
df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)

cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'TIPO_FONDO', 'clave', 'AGENCIA CORRECTA']
df_base_test = df_base_test.merge(df_asignacion_test[cols_asignacion], left_on='CC', right_on='codigo', how='left')
df_base_test = df_base_test.drop(columns=['codigo'])
df_base_test = df_base_test.rename(columns={'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO', 'nombre': 'NOMBRE_CLIENTE', 'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA'})
df_base_test.sort_values(by=['TIPO_CARTERA'], ascending=False, inplace=True)
df_base_test.sort_values(by=['CC'], ascending=True, inplace=True)

filtro = (df_base_test['FLAG'] != 1) & (df_base_test['FLAG'].notna())
df_base_filtro = df_base_test.loc[filtro].groupby('CC').apply(actualizar_tipo_cartera).reset_index(drop=True)
df_base_test = df_base_test.merge(df_base_filtro[['CC', 'TIPO_CARTERA']], on='CC', how='left', suffixes=('', '_y'))
df_base_test['TIPO_CARTERA_FINAL'] = df_base_test['TIPO_CARTERA_y'].fillna(df_base_test['TIPO_CARTERA'])
df_base_test = df_base_test.drop(columns=['TIPO_CARTERA', 'TIPO_CARTERA_y'])
df_base_test = df_base_test.rename(columns={'TIPO_CARTERA_FINAL': 'TIPO_CARTERA'})
#26497246
df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(114, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                Int64
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-02,01836935,00248,142,1000.0,PEN,MARCIAL AUGUSTO SOLIS VASQUEZ.....,P,1,001101759600256572,NULL,KSTBC,MARCIAL AUGUSTO SOLIS VASQUEZ,03-Sep,MIV,UNSECURED,CLASA MORA
1,2024-09-02,04101561,00248,87,2000.0,PEN,SANTOS MORAN CISNERON1,P,1,001102745001992697,NULL,KSTBC,SANTOS MORAN CISNERON,03-Sep,MIV,UNSECURED,CLASA MORA
2,2024-08-31,09839518,00248,215,1500.0,PEN,ROBERTO ELEODORO VELASQUEZ MORALES.,P,1,001102155001157281,NULL,BC,ROBERTO ELEODORO VELASQUEZ MORALES,03-Sep,MIV,SECURED,SIN AGENCIA
3,2024-08-31,20157603,00248,164,100.0,PEN,LUIS ARMANDO FIERRE TRUJILLO,P,1,001104085006185487,NULL,EXJ,LUIS ARMANDO FIERRO TRUJILLO,03-Sep,MIV,UNSECURED,CLASA MORA
4,2024-09-02,20275065,00248,127,150.0,PEN,MARIA EUGENIA SALAZAR SALVATIERRA,P,2,001104085009944508,NULL,KSTBC,MARIA EUGENIA SALAZAR SALVATIERRA,03-Sep,MIV,UNSECURED,MORNESE MORA


In [397]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

REGISTROS OK


In [398]:
df_base_test.shape

(114, 17)

In [399]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
df_base_no_flag.shape

(12, 17)

In [400]:
no_flag_count = df_base_no_flag.shape[0]
no_flag_count

12

In [401]:
flag_count = base_count - no_flag_count
flag_count

102

In [402]:
#################### MONOPRODUCTO ####################

In [403]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
print(df_mono.shape)
df_mono.head(5)

(85, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-02,01836935,00248,142,1000.0,PEN,MARCIAL AUGUSTO SOLIS VASQUEZ.....,P,1,001101759600256572,NULL,KSTBC,MARCIAL AUGUSTO SOLIS VASQUEZ,03-Sep,MIV,UNSECURED,CLASA MORA
1,2024-09-02,04101561,00248,87,2000.0,PEN,SANTOS MORAN CISNERON1,P,1,001102745001992697,NULL,KSTBC,SANTOS MORAN CISNERON,03-Sep,MIV,UNSECURED,CLASA MORA
2,2024-08-31,09839518,00248,215,1500.0,PEN,ROBERTO ELEODORO VELASQUEZ MORALES.,P,1,001102155001157281,NULL,BC,ROBERTO ELEODORO VELASQUEZ MORALES,03-Sep,MIV,SECURED,SIN AGENCIA
3,2024-08-31,20157603,00248,164,100.0,PEN,LUIS ARMANDO FIERRE TRUJILLO,P,1,001104085006185487,NULL,EXJ,LUIS ARMANDO FIERRO TRUJILLO,03-Sep,MIV,UNSECURED,CLASA MORA
8,2024-09-02,20303444,00248,327,1000.0,PEN,PRESTAMO 001108319601418288 MARIA VEINTE,P,1,001108319601418288,NULL,KSTBC,MARIA DEL CARMEN VEINTIMILLA RUIZ,03-Sep,MIV,UNSECURED,CLASA MORA


In [404]:
mono_count = df_mono.shape[0]
mono_count

85

In [405]:
df_mono['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    70
SECURED      14
NULL          1
Name: count, dtype: int64

In [406]:
df_mono['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        77
REACTIVA     8
Name: count, dtype: int64

In [407]:
df_mono[['TIPO_CARTERA', 'TIPO_FONDO']].value_counts()

TIPO_CARTERA  TIPO_FONDO
UNSECURED     NULL          65
SECURED       NULL          11
UNSECURED     REACTIVA       5
SECURED       REACTIVA       3
NULL          NULL           1
Name: count, dtype: int64

In [408]:
#################### MONO REACTIVA ####################

In [409]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'] != 'NULL')]
df_reactiva.sort_values(by=['CC'], inplace=True)
df_reactiva.reset_index(drop=True, inplace=True)
print(df_reactiva.shape)
df_reactiva.head(5)

(5, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-09-02,22673977,00248,370,1000.00,PEN,C Y C MADERAS Y SERVICIOS GENERALES SAC,P,1,001102339600153034,REACTIVA,KSTBC,C Y C MADERAS Y SERVICIOS GENERALES SAC,03-Sep,MIV,UNSECURED,MORNESE MORA
1,2024-09-02,24173081,00248,301,2000.00,PEN,"CAUDAL INVERSIONES EIRL,,,",P,1,001103019601138895,REACTIVA,BC,CAUDAL INVERSIONES EIRL,03-Sep,MIV,UNSECURED,CLASA MORA
2,2024-09-02,25887699,00248,30,2242.69,PEN,CUNA GUARDERIA JARDIN ESTRELLITAS EIRL,P,1,001100309600046722,REACTIVA,KSTBC,CUNA GUARDERIA JARDIN ESTRELLITAS EIRL,03-Sep,MIV,UNSECURED,CLASA MORA
3,2024-08-31,28589314,00248,321,400.00,PEN,SERVICIOS MINEROS,P,1,001103219600540265,REACTIVA,EXJ,SERVICIOS MINEROS DEL CENTRO SAC,03-Sep,MIV,UNSECURED,ASESCOM RJ
4,2024-09-02,28978925,00248,143,5000.00,PEN,"ARIANA AG EIRL .-,'",P,1,001102529600107303,REACTIVA,EXJ,ARIANA AG EIRL,03-Sep,MIV,UNSECURED,ASESCOM RJ


In [410]:
reactiva_count = df_reactiva.shape[0]
reactiva_count

5

In [411]:
df_reactiva.to_excel(reactiva, index=False)

In [412]:
#################### FIN REACTIVA ####################

In [413]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
no_mono_count

15

In [414]:
df_mono = df_mono[df_mono['TIPO_CARTERA'] == 'UNSECURED']
df_mono = df_mono[df_mono['TIPO_FONDO'] == 'NULL']

cols_mono = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA']
df_mono = df_mono[cols_mono]

df_mono.sort_values(by=['CC'], inplace=True)
df_mono.reset_index(drop=True, inplace=True)

print(df_mono.shape)
df_mono.head(5)

(65, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-09-02,01836935,00248,1000.0,PEN,MARCIAL AUGUSTO SOLIS VASQUEZ.....,P,1,001101759600256572,MARCIAL AUGUSTO SOLIS VASQUEZ,03-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA
1,2024-09-02,04101561,00248,2000.0,PEN,SANTOS MORAN CISNERON1,P,1,001102745001992697,SANTOS MORAN CISNERON,03-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA
2,2024-08-31,20157603,00248,100.0,PEN,LUIS ARMANDO FIERRE TRUJILLO,P,1,001104085006185487,LUIS ARMANDO FIERRO TRUJILLO,03-Sep,MIV,UNSECURED,NULL,EXJ,CLASA MORA
3,2024-09-02,20303444,00248,1000.0,PEN,PRESTAMO 001108319601418288 MARIA VEINTE,P,1,001108319601418288,MARIA DEL CARMEN VEINTIMILLA RUIZ,03-Sep,MIV,UNSECURED,NULL,KSTBC,CLASA MORA
4,2024-08-31,20558875,00248,150.0,PEN,AMERICO WILLY AREVALO SALDA･A.......,P,1,001103015003510296,AMERICO WILLY AREVALO SALDANA,03-Sep,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ


In [415]:
df_mono.to_excel(monoproducto, index=False)

In [416]:
mono_count_final = df_mono.shape[0]
mono_count_final

65

In [417]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

MONOPRODUCTO OK


In [418]:
#################### FIN MONOPRODUCTO ####################

In [419]:
#################### MULTIPRODUCTO ####################

In [420]:
df_multi = df_base_test[df_base_test['FLAG'] > 1].copy()
print(df_multi.shape)
df_multi.head(5)

(17, 17)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
4,2024-09-02,20275065,00248,127,150.0,PEN,MARIA EUGENIA SALAZAR SALVATIERRA,P,2,001104085009944508,NULL,KSTBC,MARIA EUGENIA SALAZAR SALVATIERRA,03-Sep,MIV,UNSECURED,MORNESE MORA
21,2024-09-02,21298267,00248,231,2000.0,PEN,BRUNO ARMANDO BACA G.,P,2,001105045002450188,NULL,KSTBC,BRUNO ARMANDO BACA GOMEZ,03-Sep,MIV,UNSECURED,CLASA MORA
40,2024-09-02,22312384,00248,857,2814.0,PEN,AIMEE LESLIE JIMENEZ ARAUJO.,P,2,001101339600216458,NULL,EXJ,AIMEE LESLIE JIMENEZ ARAUJO,03-Sep,MIV,UNSECURED,CLASA MORA
52,2024-08-31,22826478,00248,281,5500.0,PEN,42388743,P,2,001108149603537931,NULL,KSTBC,MARLO ALBERTO PEREZ JIMENEZ,03-Sep,MIV,UNSECURED,CLASA MORA
64,2024-09-02,24038303,00248,50,6000.0,PEN,LADRILLERIA PARANAGUA SU SRL,P,2,001103019601140385,REACTIVA,BC,LADRILLERA PARANAGUASU SRL,03-Sep,MIV,SECURED,SIN AGENCIA


In [421]:
multi_count = df_multi.shape[0]
multi_count

17

In [422]:
df_multi['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    12
SECURED       5
Name: count, dtype: int64

In [423]:
df_multi['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        14
REACTIVA     3
Name: count, dtype: int64

In [424]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
no_multi_count

5

In [425]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

df_multi = df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'])

df_multi.sort_values(by=['CC'], inplace=True)
df_multi.reset_index(drop=True, inplace=True)

print(df_multi.shape)
df_multi.head(5)

(12, 14)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
0,2024-09-02,20275065,00248,150.0,PEN,MARIA EUGENIA SALAZAR SALVATIERRA,P,2,None,None,None,MARIA EUGENIA SALAZAR SALVATIERRA,03-Sep,MIV
1,2024-09-02,21298267,00248,2000.0,PEN,BRUNO ARMANDO BACA G.,P,2,None,None,None,BRUNO ARMANDO BACA GOMEZ,03-Sep,MIV
2,2024-09-02,22312384,00248,2814.0,PEN,AIMEE LESLIE JIMENEZ ARAUJO.,P,2,None,None,None,AIMEE LESLIE JIMENEZ ARAUJO,03-Sep,MIV
3,2024-08-31,22826478,00248,5500.0,PEN,42388743,P,2,None,None,None,MARLO ALBERTO PEREZ JIMENEZ,03-Sep,MIV
4,2024-09-02,24207146,00248,800.0,PEN,"EVELYN MENDOZA ROSALES,,",P,2,None,None,None,EVELYN JASMIN MENDOZA ROSALES,03-Sep,MIV


In [426]:
df_multi.to_excel(multiproducto, index=False)

In [427]:
multi_count_final = df_multi.shape[0]
multi_count_final

12

In [428]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

MULTIPRODUCTO OK


In [429]:
#################### FIN MULTIPRODUCTO ####################

In [430]:
#################### NO ENVIADOS ####################

In [431]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

No enviados: 32
Enviados: 82
Total: 114


In [432]:
if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

REGISTROS OK


In [433]:
print(df_no_mono.columns)
print(df_no_multi.columns)
print(df_base_no_flag.columns)

Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')
Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')
Index(['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE',
       'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE',
       'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA'],
      dtype='object')


In [434]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].astype('Int64').fillna(0)

df_no_enviados['CONTRATO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['CONTRATO'], axis=1)
df_no_enviados['TIPO_FONDO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_FONDO'], axis=1)
df_no_enviados['TIPO_CARTERA'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_CARTERA'], axis=1)

df_no_enviados['FLAG'] = df_no_enviados['FLAG'].apply(lambda x: '' if x == 0 else x)

df_no_enviados.sort_values(by=['CC'], inplace=True)
df_no_enviados.reset_index(drop=True, inplace=True)

print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

(32, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG               object
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-08-31,09839518,00248,215,1500.0,PEN,ROBERTO ELEODORO VELASQUEZ MORALES.,P,1,001102155001157281,NULL,BC,ROBERTO ELEODORO VELASQUEZ MORALES,03-Sep,MIV,SECURED,SIN AGENCIA
1,2024-09-02,20613626,00248,136,10000.0,PEN,MARIA PATRICIA CERNA MONJE.,P,1,001104869600914554,NULL,BC,MARIA PATRICIA CERNA MONJE,03-Sep,MIV,SECURED,SIN AGENCIA
2,2024-09-02,20935081,00248,358,75.0,PEN,001104085006730582,P,,None,None,NaN,NaN,03-Sep,MIV,None,NaN
3,2024-08-31,20982291,00248,177,2500.0,PEN,"JORGE ARMANDO GARCIA BARRIOS,1/",P,1,001108319601006834,NULL,BC,JORGE ARMANDO GARCIA BARRIOS,03-Sep,MIV,SECURED,SIN AGENCIA
4,2024-09-02,21403390,00249,148,3000.0,USD,"IZQUIERDO & CASAFRAN CA CONSTRUCCIONES,,",P,1,001103979600067823,NULL,BC,IZQUIERDO & CASAFRANCA CONSTRUCCIONES METALICA...,03-Sep,MIV,SECURED,SIN AGENCIA


In [435]:
if no_eviados == df_no_enviados.shape[0]:
    print('NO ENVIADOS OK')
else:
    print('NO ENVIADOS ERROR')

NO ENVIADOS OK


In [436]:
df_no_enviados.to_excel(no_enviados_path, index=False)

In [437]:
#################### FORMATO EXCEL ####################

In [438]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 15):  # Columnas L-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'instruccion':
        for col in range(1, 11): # Columnas A-J
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(11, 13): # Columnas K-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 17):  # Columnas M-P
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(17, 18):  # Columna Q
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

In [439]:
format_excel(mono_path, "mono")
format_excel(multi_path, "multi")
format_excel(react_path, "instruccion")
format_excel(no_enviados_path, "instruccion")

os.startfile(mono_path)
os.startfile(multi_path)
os.startfile(react_path)        
os.startfile(no_enviados_path)

In [440]:
#################### AGENCIAS ####################

In [441]:
#df_1 = pd.read_excel('agencias/RJ.xlsx')
#df_2 = pd.read_excel('agencias/CLASA.xlsx')
#df_3 = pd.read_excel('agencias/MORNESE.xlsx')

#df_1['AGENCIA'] = df_1['AGENCIA'].fillna('ASESCOM RJ')
#df_2['AGENCIA'] = df_2['AGENCIA'].fillna('CLASA MORA')
#df_3['AGENCIA'] = df_3['AGENCIA'].fillna('MORNESE MORA')

#df_1['CONTRATO'] = df_1['CONTRATO'].fillna('sin contrato')
#df_2['CONTRATO'] = df_2['CONTRATO'].fillna('sin contrato')
#df_3['CONTRATO'] = df_3['CONTRATO'].fillna('sin contrato')


In [442]:
#df_test = pd.read_excel('reporte/2024.09.03/MULTIPRODUCTO 02.09.2024.xlsx')
#print(df_test.shape)
#df_test.head(5)

In [443]:
#df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['cont_18'] = df_asignacion_test['cont_18'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_test['CONTRATO'] = df_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

df_test = df_test.drop(columns=['TIPO_FONDO', 'CARTERA'])

cols_asignacion = ['cont_18', 'TIPO_FONDO', 'clave', 'TIPO_CARTERA', 'AGENCIA CORRECTA']
df_test = df_test.merge(df_asignacion_test[cols_asignacion], left_on='CONTRATO', right_on='cont_18', how='left')

df_test = df_test.drop(columns=['cont_18'])

df_test = df_test.rename(columns={'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA'})

cols_multi_final = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_test = df_test[cols_multi_final]

print(df_test.shape)
print(df_test.dtypes)
df_test.head(25)

NameError: name 'df_test' is not defined

In [ ]:
#df_test.to_excel('reporte/2024.09.03/MULTIPRODUCTO 02.09.2024_FINAL.xlsx', index=False)

In [ ]:
#format_excel('reporte/2024.09.03/MULTIPRODUCTO 02.09.2024_FINAL.xlsx', "multi")